In [ ]:
#CNS-Ac for Chen[0,500] 

In [ ]:
setprecision(3000);

In [ ]:
function accuracy(t)
    e = ep*exp(BigFloat(2.3025851)*(BigFloat(1.001)*t-BigFloat(20)))
    ac = (0.29803-log10(e))/0.30103;
    return max(round(Int,ac),128);
end

In [ ]:
function order()
    M=450;
    return M+1;
end

In [ ]:
function Ep(t)
    return ep*exp(BigFloat(2.3025851)*(BigFloat(1.001)*t-BigFloat(15)))
end

In [ ]:
function steps(Dfm,Co)
    h = Co*abs(Dfm)^(BigFloat(-1)/BigFloat(order()-1))
    return min(h,BigFloat(1)/BigFloat(10))
end

In [ ]:
function goonm(mor)
    sgo = BigFloat(1)
    for i in 1:mor
        sgo = sgo*BigFloat(i)
    end
    return (sgo)^(BigFloat(1)/BigFloat(mor))
end 

In [ ]:
function goon(mor,ep)
    sgo = BigFloat(1)
    for i in 1:mor
        sgo = sgo*BigFloat(i)
    end
    return (ep*sgo)^(BigFloat(1)/BigFloat(mor))
end 

In [ ]:
function JCh(k,i)
     
    k = BigFloat(k);
    i = BigFloat(i);
    s = BigFloat(1);
    if i < k/2
        i = k - i;
    end
    for j in range(k,i+1,step=-1)
        
        j1=BigFloat(j)
        s = s*j1/(j1-i)
        
    end
    
    return s
end

In [ ]:
function DJCh()
    Cki = Array{BigFloat}(undef,order()-1,order()-1);
    for k in 1:order()-1
        for j in 1:k
            Cki[k,j] = 1
        end
    end
    
    for k in 2:order()-2
        for i in 1:k-1
           Cki[k+1,i+1] = Cki[k,i] + Cki[k,i+1]
        end
    end
    return Cki
end

In [ ]:
function multi(hstep)
    S = Array{BigFloat}(undef,1,order());
    S[1] = BigFloat(1)
    
    for i in 1:order()-1
        S[i+1] = S[i]*hstep/BigFloat(i)
    end
    
    return S
end

In [ ]:
fdot(x,y,z) = @. x*y*z
function Sumxyz(x,z,Cki)
    return sum(fdot(Cki[1:end],reverse!(x[1:end]),z[1:end]))
end 

In [ ]:
function Talor(x,y,z,Cki,Co)
    for k in 1:order()-1
        x[k+1] = a*(y[k] - x[k])
        y[k+1] = (c-a)*x[k] - Sumxyz(x[1:k],z[1:k],Cki[k,1:k]) + c*y[k]
        z[k+1] = Sumxyz(x[1:k],y[1:k],Cki[k,1:k]) - b*z[k]
    end
    
    return x,y,z,min(steps(x[end],Co),steps(y[end],Co),steps(z[end],Co))
end

In [ ]:
function Iteration(x,y,z,Cki,Tin,Tinf)
    stl = 0;
    for st in 1:Times
    
        Co = Mo*Ep(Tin)^(BigFloat(1)/BigFloat(order()))
        setprecision(accuracy(Tin))
        
        x,y,z,hstep = Talor(x,y,z,Cki,Co)
        
        if Tin + hstep < Tinf
            
            Dmulti = multi(hstep);
            x[1] = sum(x[1:end].*Dmulti[1:end])
            y[1] = sum(y[1:end].*Dmulti[1:end])
            z[1] = sum(z[1:end].*Dmulti[1:end])
            Lorenz[st,:] = [x[1],y[1],z[1]] 
            Lorenzf[st,:] = convert.(Float32,Lorenz[st,:])
        
            Tin = Tin + hstep
            if mod(st,500) == 0
            #println(st,")","Time=",convert.(Float16,Tin),": ",Lorenzf[st,:]," (Ep: ",convert(Float16,log10(abs(Ep(Tin)))),") (Mp: ",accuracy(Tin),")")
            println(convert.(Float16,Tin))
            end
        else
            hstep = Tinf - Tin
            Dmulti = multi(hstep);
            x[1] = sum(x[1:end].*Dmulti[1:end])
            y[1] = sum(y[1:end].*Dmulti[1:end])
            z[1] = sum(z[1:end].*Dmulti[1:end])
            Lorenz[st,:] = [x[1],y[1],z[1]] 
            Lorenzf[st,:] = convert.(Float32,Lorenz[st,:])
        
            Tin = Tinf
            println(st,")","Time=",convert.(Float16,Tin),": ",Lorenzf[st,:]," (Ep: ",convert(Float16,log10(abs(Ep(Tin)))),") (Mp: ",accuracy(Tin),")")
            #println(convert.(Float16,Tin))
            stl = st
            break;
        end
        
    end
    
    return Lorenz,stl
    
end

In [ ]:
Times = 1500000;      
Tin = BigFloat(0);
Tinf = BigFloat(500);

ep = BigFloat(10)^(-20-1.001*Tinf)
Mo = goonm(order())

a =  BigFloat(40);
b =  BigFloat(3);
c =  BigFloat(28)

x = Array{BigFloat}(undef,order())
y = Array{BigFloat}(undef,order())
z = Array{BigFloat}(undef,order())

Lorenz = Array{BigFloat}(undef,Times,3);
Ros = Array{BigFloat}(undef,Times,3);
Lorenzf = Array{Float32}(undef,Times,3);
Dmulti = Array{BigFloat}(undef,1,order());
Cki = Array{BigFloat}(undef,order()-1,order()-1);

x[1] = -BigFloat(1)/BigFloat(10); 
y[1] = BigFloat(1)/BigFloat(2); 
z[1] = -BigFloat(6)/BigFloat(10);

Cki = DJCh();
println("ep0: ",convert(Float16,log10(abs(ep))))

In [ ]:
Ros,st = Iteration(x,y,z,Cki,Tin,Tinf);

In [ ]:
using Plots; 
plot(Ros[1:st,1],Ros[1:st,2],Ros[1:st,3])